In [1]:
# Import required libraries.

import pandas as pd
import numpy as np
import time

In [2]:
t_begin = time.time()
t_read = time.time()

In [3]:
# Read all datasets.

jan  = pd.read_csv("datasets/01_flight_details_january.csv")
feb  = pd.read_csv("datasets/02_flight_details_february.csv")
mar  = pd.read_csv("datasets/03_flight_details_march.csv")
apr  = pd.read_csv("datasets/04_flight_details_april.csv")
may  = pd.read_csv("datasets/05_flight_details_may.csv")
jun  = pd.read_csv("datasets/06_flight_details_june.csv")
jul  = pd.read_csv("datasets/07_flight_details_july.csv")
aug  = pd.read_csv("datasets/08_flight_details_august.csv")
sep  = pd.read_csv("datasets/09_flight_details_september.csv")
octo = pd.read_csv("datasets/10_flight_details_october.csv")
nov  = pd.read_csv("datasets/11_flight_details_november.csv")
dec  = pd.read_csv("datasets/12_flight_details_december.csv")


In [4]:
t_read_complete = time.time() - t_read

In [5]:
# Make list of dasets to make iterating easy.

df_list = [jan, feb, mar, apr, may, jun, jul, aug, sep, octo, nov, dec]

In [6]:
jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450017 entries, 0 to 450016
Data columns (total 25 columns):
FL_DATE                450017 non-null object
AIRLINE_ID             450017 non-null int64
TAIL_NUM               449378 non-null object
FL_NUM                 450017 non-null int64
ORIGIN                 450017 non-null object
ORIGIN_CITY_NAME       450017 non-null object
DEST                   450017 non-null object
DEST_CITY_NAME         450017 non-null object
CRS_DEP_TIME           450017 non-null int64
DEP_TIME               441476 non-null float64
DEP_DELAY              441476 non-null float64
DEP_DELAY_GROUP        441476 non-null float64
CRS_ARR_TIME           450017 non-null int64
ARR_TIME               440746 non-null float64
ARR_DELAY              439645 non-null float64
ARR_DELAY_GROUP        439645 non-null float64
CANCELLED              450017 non-null float64
DIVERTED               450017 non-null float64
AIR_TIME               439645 non-null float64
CARRIER_DE

In [7]:
# Make copies of datasets.
df_copy_list = []
for df in df_list:
    df_copy_list.append(df.copy())

# Deconstruct copy list for redable access to elements.
jan_copy, feb_copy, mar_copy, apr_copy, may_copy, jun_copy, jul_copy, aug_copy, sep_copy, octo_copy, nov_copy, dec_copy = df_copy_list

In [8]:
t_convert = time.time()

In [9]:
# Convert columns to appropriate datatypes.

for df in df_copy_list:
        
    df.FL_DATE = pd.to_datetime(df.FL_DATE)
    
    df.AIRLINE_ID = df.AIRLINE_ID.astype("str")
    
    df.FL_NUM = df.FL_NUM.astype("str")
    
    df.CRS_DEP_TIME = df.CRS_DEP_TIME.astype("str").str.zfill(4).replace("2400", "2359")
    df.CRS_ARR_TIME = df.CRS_ARR_TIME.astype("str").str.zfill(4).replace("2400", "2359")    
    df.CRS_DEP_TIME = pd.to_datetime(df.CRS_DEP_TIME, format="%H%M").dt.time
    df.CRS_ARR_TIME = pd.to_datetime(df.CRS_ARR_TIME, format="%H%M").dt.time
    
    df.DEP_TIME = df.DEP_TIME.astype("int", errors="ignore").astype("str").str.zfill(4).replace("2400", "2359")
    df.ARR_TIME = df.ARR_TIME.astype("int", errors="ignore").astype("str").str.zfill(4).replace("2400", "2359")
    
    df.DEP_DELAY_GROUP = df.DEP_DELAY_GROUP.astype("int", errors="ignore")
    df.ARR_DELAY_GROUP = df.ARR_DELAY_GROUP.astype("int", errors="ignore")
    
    df.DEP_DELAY = df.DEP_DELAY.astype("int", errors="ignore")
    df.ARR_DELAY = df.ARR_DELAY.astype("int", errors="ignore")
    
    df.CANCELLED = df.CANCELLED.astype("bool")
    df.DIVERTED = df.DIVERTED.astype("bool")
    
    df.TAIL_NUM = df.TAIL_NUM.replace(np.NaN, "CANCELLED")
    df["FLIGHT_ID"] = df.index.astype("str").str.zfill(7) + "-" + df["AIRLINE_ID"] + df["TAIL_NUM"]
    
    df.drop(["AIRLINE_ID", "TAIL_NUM", "Unnamed: 24"], axis=1, inplace=True)
    
    df[["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].replace(0, np.NaN)
    
    df["ORIGIN_CITY"], df["ORIGIN_STATE"] = df.ORIGIN_CITY_NAME.str.split(", ").str
    df["DEST_CITY"], df["DEST_STATE"] = df.DEST_CITY_NAME.str.split(", ").str
    df.drop(["ORIGIN_CITY_NAME", "DEST_CITY_NAME"], inplace=True, axis=1)
    
    df[["CARRIER_DELAY", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]].replace(0, np.NaN)

In [10]:
t_convert_complete = time.time() - t_convert

In [11]:
t_status = time.time()

In [12]:
count = 0
for df in df_copy_list:
    fls = []
    for index, flight in df.iterrows():
        if flight.CANCELLED:
            FLIGHT_STATUS = "Cancelled"
        elif flight.DIVERTED:
            FLIGHT_STATUS = "Diverted"
        else:
            FLIGHT_STATUS = "Normal"
        
        fls.append({"FLIGHT_ID": flight.FLIGHT_ID,
                    "FLIGHT_STATUS": FLIGHT_STATUS})
    
    fls_df = pd.DataFrame(fls)
    df_copy_list[count] = df.merge(fls_df, on="FLIGHT_ID", how="left")
    count += 1
    
jan_copy, feb_copy, mar_copy, apr_copy, may_copy, jun_copy, jul_copy, aug_copy, sep_copy, octo_copy, nov_copy, dec_copy = df_copy_list

In [13]:
t_status_complete = time.time() - t_status
t_end = time.time() - t_begin

In [14]:
print("Read: {}\nConvert: {}\nStatus: {}\nTotal: {}".format(t_read_complete, t_convert_complete, t_status_complete, t_end))

Read: 14.246224641799927
Convert: 119.45770001411438
Status: 752.5466537475586
Total: 887.0517683029175


In [18]:
jan_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450017 entries, 0 to 450016
Data columns (total 24 columns):
FL_DATE                450017 non-null datetime64[ns]
FL_NUM                 450017 non-null object
ORIGIN                 450017 non-null object
DEST                   450017 non-null object
CRS_DEP_TIME           450017 non-null object
DEP_TIME               450017 non-null object
DEP_DELAY              441476 non-null float64
DEP_DELAY_GROUP        441476 non-null float64
CRS_ARR_TIME           450017 non-null object
ARR_TIME               450017 non-null object
ARR_DELAY              439645 non-null float64
ARR_DELAY_GROUP        439645 non-null float64
AIR_TIME               439645 non-null float64
CARRIER_DELAY          97699 non-null float64
WEATHER_DELAY          97699 non-null float64
NAS_DELAY              97699 non-null float64
SECURITY_DELAY         97699 non-null float64
LATE_AIRCRAFT_DELAY    97699 non-null float64
FLIGHT_ID              450017 non-null object
OR

In [17]:
for df in df_copy_list:
    df.drop(["CANCELLED", "DIVERTED"], axis=1, inplace=True)

In [19]:
master_df = pd.concat(df_copy_list)

In [ ]:
master_df.to_csv("datasets/2018_claened.csv", index=False)